# Training Models

In [1]:
import time
import joblib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay

In [2]:
df = pd.read_csv("handSign8.csv")

In [ ]:
df["target"].value_counts().sort_index()

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Support Vector Machine

In [ ]:
# svm
start = time.time()
svm = SVC(C=100, gamma=0.1, kernel='rbf', probability=True).fit(X_train, y_train)
svm.score(X_test, y_test), svm.score(X_train, y_train), svm.score(X, y)
end = time.time()
print(end-start)

In [6]:
svm = joblib.load("new models/svm_advanced_2.pkl")

In [ ]:
joblib.dump(svm, 'new models/svm_advanced_2.pkl')

## K Neighbors Classifier

In [ ]:
start = time.time()
knn = KNeighborsClassifier(n_neighbors=2, algorithm='auto', weights='distance', n_jobs=-1).fit(X_train, y_train)
knn.score(X_test, y_test), knn.score(X_train, y_train), knn.score(X, y)
end = time.time()
print(end-start)

In [7]:
knn = joblib.load("new models/knn_advanced.pkl")

In [ ]:
joblib.dump(knn, 'new models/knn_advanced.pkl')

## Decisoin Tree Classifier

In [ ]:
start = time.time()
dt = DecisionTreeClassifier(criterion='entropy', splitter='random').fit(X_train, y_train)
dt.score(X_test, y_test), dt.score(X_train, y_train), dt.score(X, y)
end = time.time()
print(end-start)

In [8]:
dt = joblib.load("new models/dt_advanced.pkl")

In [ ]:
joblib.dump(dt, 'new models/dt_advanced.pkl')

## Random Forest Classifier

In [ ]:
# Initialize the Random Forest classifier
start = time.time()
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf.fit(X_train, y_train)
end = time.time()
print(end-start)

In [9]:
rf = joblib.load("new models/rf_advanced.pkl")

In [ ]:
rf.score(X_test, y_test), dt.score(X_train, y_train), dt.score(X, y)

In [ ]:
joblib.dump(dt, 'new models/rf_advanced.pkl')

## Voting Classifier

### Hard

In [ ]:
start = time.time()
eclf = VotingClassifier(estimators=[
    ('svm', svm), 
    ('dt', dt), 
    ('rf', rf),
    ('knn', knn)
], voting='hard')

# Train the VotingClassifier
eclf.fit(X_train, y_train)
end = time.time()
print(end-start)

In [ ]:
eclf.score(X_test, y_test), eclf.score(X_train, y_train), eclf.score(X, y)

In [ ]:
joblib.dump(eclf, 'new models/voting_hard_advanced.pkl')

### Soft

In [ ]:
start = time.time()
eclf = VotingClassifier(estimators=[
    ('svm', svm), 
    ('dt', dt), 
    ('rf', rf),
    ('knn', knn)
], voting='soft')

# Train the VotingClassifier
eclf.fit(X_train, y_train)
end = time.time()
eclf.score(X_test, y_test), eclf.score(X_train, y_train), eclf.score(X, y)
print(end-start)

In [ ]:
joblib.dump(eclf, 'new models/voting_soft_advanced.pkl')

# Making Confusion Matrix

In [21]:
def metrics(y_true, y_pred, average = 'weighted'):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average=average)
    recall = recall_score(y_true, y_pred, average=average)
    f1 = f1_score(y_true, y_pred, average=average)
    return {"accuracy" : accuracy, "precision" : precision, "recall" : recall, "f1_score" : f1}

In [ ]:
# k neghbors 
y_pred = knn.predict(X)
metrics(y, y_pred)

In [ ]:
# support vector machine
y_pred = svm.predict(X)
metrics(y, y_pred)

In [ ]:
# decision tree classifier
y_pred = dt.predict(X)
metrics(y, y_pred)

In [ ]:
# display confusion matrix
ConfusionMatrixDisplay(confusion_matrix(y, y_pred)).plot()

In [ ]:
# To save confusion matrix as png

import matplotlib
import seaborn as sns
cm = confusion_matrix(y, y_pred)
matplotlib.use('Agg')
plt.figure(figsize=(15, 10))
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5)
plt.show()
# plt.savefig("confusion.png")